<a href="https://colab.research.google.com/github/waleedsial/Spark---Data-Engineering-/blob/master/Spark_Transformations_%26_Actions_With_CoronaVirus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls /usr/lib/jvm

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [0]:

# Installing JVM
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Installing spark from spark website. 
!wget -q https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

# Unzipping 
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

# Let us allow to find the spark & set the path variable 
!pip install -q findspark


In [0]:
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
import findspark
# finding spark & setting system path 
findspark.init()
from pyspark.sql import SparkSession
# In order to work in spark we need spark context 

# We are setting local since we dont have distributed nodes
spark = SparkSession.builder.master("local[*]").getOrCreate()




In [0]:
import sys, tempfile, urllib
from pyspark.sql.functions import * 


In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
# urllib.request.urlretrieve(url, filename=None, reporthook=None, data=None)
BASE_DIR = '/tmp'
CORONA_DATA_FILE = os.path.join(BASE_DIR, 'corona_data.csv')
corona_data = urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/corona_dataset_latest.csv',CORONA_DATA_FILE )


In [0]:
BASE_DIR = '/tmp'
TWITTER_DATA_FILE = os.path.join(BASE_DIR, 'twitter_data.csv')
tweets_data = urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/coronavirus/tweets.csv',TWITTER_DATA_FILE )

In [11]:
!ls /tmp

blockmgr-888b6dff-7d7f-46f9-8d42-98b23ce4fa0c
corona_data.csv
hsperfdata_root
spark-6da23c70-6e8d-4370-af08-c316f4ae4966
spark-f4ca39a6-9d54-4238-a167-7b7dc083925a
twitter_data.csv


In [0]:
# Creating spark dataframe 
# if we have a file we can use read option 
# We are telling to read the file & infer schema based on the file 
# header 
corona_df = spark.read.option("inferSchema", "true").csv("/tmp/corona_data.csv", header = True)
twitter_df = spark.read.option("inferSchema", "true").csv("/tmp/twitter_data.csv", header = True)

In [15]:
type(corona_df)
type(twitter_df)

pyspark.sql.dataframe.DataFrame

pyspark.sql.dataframe.DataFrame

In [22]:
corona_df.count()

28143

In [23]:


corona_df.show()
# Teh data follows a time series pattern


+---+----------------+--------------------+--------+---------+-------------------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|               Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+-------------------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22 00:00:00|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22 00:00:00|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22 00:00:00|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22 00:00:00|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|   

In [24]:

twitter_df.show()

+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0| geo|                text|                user|            location|            entities|           sentiment|             country|
+---+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|null|What is God sayin...|          petodinice|               Lagos|[('about #', 'CAR...|{'neg': 0.0, 'neu...|             Nigeria|
|  1|null|"BREAKING: ""this...| but i took the t...|                -… "|     JerryfranksonJF|      Abuja, Nigeria|"[(""Arsenal's Mi...|
|  2|null| #Coronavirus tes...|              cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|                 USA|
|  3|null| Get ready for ma...|        InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|                 USA|
|  4|null| The #coronavirus...|          

Transformations - Raw Data to Transformations 
Aggregations 
Transformation creates new DF by DAG. 
The plan only executes once action is called. Spark optimizes this plan. 

In [26]:
twitter_df.filter("country = 'USA'")
# This tells us that a dataframe will be created, it is not created yet. 

DataFrame[_c0: int, geo: string, text: string, user: string, location: string, entities: string, sentiment: string, country: string]

In [27]:
# Here the transformations is completed because we haev called action. 
twitter_df.filter("country = 'USA'").show()

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|  2|null| #Coronavirus tes...|         cek422|   Pennsylvania, USA|                  []|{'neg': 0.173, 'n...|    USA|
|  3|null| Get ready for ma...|   InfectiousDz|                 NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|    USA|
|  5|null| COVID-19 update ...| StewartNgilana|Durban | Port Eli...|[('Italy', 'GPE')...|{'neg': 0.178, 'n...|    USA|
|  6|null| It’s painful to ...|      BWheatnyc|             Florida|                  []|{'neg': 0.098, 'n...|    USA|
|  8|null| Questions about ...|    straightj23|        Columbus, OH|[('NAfME', 'CARDI...|{'neg': 0.0, 'neu...|    USA|
| 13|null| We’re the heck w...| harrytiffanyiv| 

In [28]:
# We can apply different conditions inside the filter 

twitter_df.filter("country = 'USA' and location like '%New%'").show()
# here we see the location is now further filtered. 

+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
|_c0| geo|                text|           user|            location|            entities|           sentiment|country|
+---+----+--------------------+---------------+--------------------+--------------------+--------------------+-------+
| 31|null| I ordered Alex J...|       rcgillan|       New York, USA|[('Alex Jones', '...|{'neg': 0.109, 'n...|    USA|
| 49|null| This week we are...|  JamesWithers3|  New York, New York|[('This week', 'D...|{'neg': 0.0, 'neu...|    USA|
| 62|null|Fear will kill #C...| RobertPPurcell|norther New Jerse...|                ['']|{'neg': 0.798, 'n...|    USA|
|228|null|This is a very co...|baskingntheGlow|       New York City|[('hourly', 'TIME')]|{'neg': 0.12, 'ne...|    USA|
|238|null|I’m reposting thi...|   Veronicaromm|     New Jersey, USA|[('English', 'LAN...|{'neg': 0.0, 'neu...|    USA|
|261|null|Too early ...?  #...|      HJeppesen| 

# How the execution plan is created. 
we can see that Spark is building up a plan for how it will execute this across the cluster by looking
at the explain plan.
We can call explain on any DataFrame object to see the DataFrame’s
lineage (or how Spark will execute this query)


In [29]:


tw_filter_df = twitter_df.filter("country = 'USA'")
tw_filter_df.explain()

# This comes from bottom to top
# read the file 
# apply the filter 
# it adds the filter condition as required. 

== Physical Plan ==
*(1) Project [_c0#42, geo#43, text#44, user#45, location#46, entities#47, sentiment#48, country#49]
+- *(1) Filter (isnotnull(country#49) && (country#49 = USA))
   +- *(1) FileScan csv [_c0#42,geo#43,text#44,user#45,location#46,entities#47,sentiment#48,country#49] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(country), EqualTo(country,USA)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...


In [30]:
tw_filter_df = twitter_df.filter(col("location").startswith("N"))
tw_filter_df.explain()
# we see in the 2nd line that the plan puts the conditions in the filter.

== Physical Plan ==
*(1) Project [_c0#42, geo#43, text#44, user#45, location#46, entities#47, sentiment#48, country#49]
+- *(1) Filter (isnotnull(location#46) && StartsWith(location#46, N))
   +- *(1) FileScan csv [_c0#42,geo#43,text#44,user#45,location#46,entities#47,sentiment#48,country#49] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/tmp/twitter_data.csv], PartitionFilters: [], PushedFilters: [IsNotNull(location), StringStartsWith(location,N)], ReadSchema: struct<_c0:int,geo:string,text:string,user:string,location:string,entities:string,sentiment:strin...


In [32]:
tw_filter_df.first()

Row(_c0=3, geo=None, text=' Get ready for mass event crowd cancellations across the World starting this weekend: cricket in #Australia in empty st…', user='InfectiousDz', location='NYC', entities="[('World', 'ORG'), ('this weekend', 'DATE'), '']", sentiment="{'neg': 0.085, 'neu': 0.798, 'pos': 0.117, 'compound': 0.1779}", country='USA')

In [33]:
tw_filter_df.take(1)


[Row(_c0=3, geo=None, text=' Get ready for mass event crowd cancellations across the World starting this weekend: cricket in #Australia in empty st…', user='InfectiousDz', location='NYC', entities="[('World', 'ORG'), ('this weekend', 'DATE'), '']", sentiment="{'neg': 0.085, 'neu': 0.798, 'pos': 0.117, 'compound': 0.1779}", country='USA')]

In [34]:
tw_filter_df.show(1)
# Show is very good boy 

+---+----+--------------------+------------+--------+--------------------+--------------------+-------+
|_c0| geo|                text|        user|location|            entities|           sentiment|country|
+---+----+--------------------+------------+--------+--------------------+--------------------+-------+
|  3|null| Get ready for ma...|InfectiousDz|     NYC|[('World', 'ORG')...|{'neg': 0.085, 'n...|    USA|
+---+----+--------------------+------------+--------+--------------------+--------------------+-------+
only showing top 1 row



In [0]:
# take, action, first are action variables. 

What are transformations ? 

To “change” a DataFrame, you need to instruct Spark how you would like to
modify it to do what you want. These instructions are called transformations.Spark will not act on transformations until we call an action. Transformations are the core of how you express your business logic using Spark.

#  Narrow & Wide Transformations 

Spark Book Page 25 

To allow every executor to perform work in parallel, Spark breaks up the data into chunks called partitions.

Transformations are simply ways of specifying different series of data
manipulation.

## Narrow 
Transformations consisting of narrow dependencies (we’ll call them narrow transformations) are those for which each input partition will contribute to only one output partition.With narrow transformations, Spark will
automatically perform an operation called pipelining, meaning that if we specify multiple filters on DataFrames, they’ll all be performed in-memory


*   All elements thare required for computation are in a single partition, for example map filter, lets say if we are applying a map and filter on each partition & creates one partition for each input. 
*   Also in case when we are mapping input from multiple partitions to one partition even than it is narrow partition. 




## Wide 

A wide dependency (or wide transformation) style transformation will have input partitions
contributing to many output partitions. You will often hear this referred to as a shuffle whereby
Spark will exchange partitions across the cluster. In case of wide transformations, spark dopes not apply all the filters in a pipeline fashion. In this case, spark writes results to the disk. 


*   For example in case of group by we need to create multiple child rdds, for example lets say there were 100 records in one partition belonging to 4,5 groups when we grouped the records on these gropups we will create 4,5 new rdds one for each group where the results from other child rdds will come as well. I am just saying this theorectically this would happen. 
*   List item








Lets see some examples of RDD functions for understanding 
sometimes going to RDD level gives us more flexibility

In [38]:
x = spark.sparkContext.parallelize([1,4,8])
# paralleize will create a parallel RDD 

# 
y = x.map(lambda x: (x, x*x))
print(x.collect())
print(y.collect())

# map is a 1-1 transformations 
# each element of a collection transformed into an element of resulting collection 

[1, 4, 8]
[(1, 1), (4, 16), (8, 64)]


In [37]:
# flatmap is not creating a tuple for each 
x = spark.sparkContext.parallelize([1,4,8])
y = x.flatMap(lambda x: (x, x*x))
print(x.collect())
print(y.collect())

# Flaty 

[1, 4, 8]
[(1, 1), (4, 16), (8, 64)]


In [39]:
x
# here x is a RDD 

ParallelCollectionRDD[114] at parallelize at PythonRDD.scala:195

In [40]:
# Here we are just selecting one column 
# we can create subset of columns as well. 
twitter_df.select("text").show()

+--------------------+
|                text|
+--------------------+
|What is God sayin...|
|"BREAKING: ""this...|
| #Coronavirus tes...|
| Get ready for ma...|
| The #coronavirus...|
| COVID-19 update ...|
| It’s painful to ...|
| 📽️Friends, I wi...|
| Questions about ...|
|How they’re deali...|
| BREAKING: Democr...|
| “If we close dow...|
| I pity the poor ...|
| We’re the heck w...|
| I don't think sh...|
| Well written, so...|
| 1/2 CDC Director...|
| In all seriousne...|
| Wash your hands....|
| #CoronaVirusCana...|
+--------------------+
only showing top 20 rows



In [41]:
# Understanding RDD , map flat map 

# converting the df in to an RDD in case if we need to do it 
# creating a map 
# splitting the line based on space 
# This is basically tokenizing 
twitter_df.rdd.map(lambda line: line.text.split(" ")).take(2)

# tokenizng each row of the column, mapping it to an array of tokens. 

[['What', 'is', 'God', 'saying', 'to', 'us', 'about', '#coronavirus', '?'],
 ['"BREAKING:', '""this', 'is', 'disappointing']]

In [44]:
# flat map on the other hand will not create this mapping of 1 to 1 
# it will flatten into one record, array 
twitter_df.rdd.flatMap(lambda line: line.text.split(" ")).take(10)
# here the take will work on the tokens , it is one to many, taking all records and flattening it out 

['What',
 'is',
 'God',
 'saying',
 'to',
 'us',
 'about',
 '#coronavirus',
 '?',
 '"BREAKING:']

In [46]:
corona_df.show()

+---+----------------+--------------------+--------+---------+-------------------+---------+-----+---------+--------------------+----+
|_c0|           State|             Country|     Lat|     Long|               Date|Confirmed|Death|Recovered|       state_cleaned|City|
+---+----------------+--------------------+--------+---------+-------------------+---------+-----+---------+--------------------+----+
|  0|            null|            Thailand|    15.0|    101.0|2020-01-22 00:00:00|        2|    0|        0|             Bangkok|null|
|  1|            null|               Japan|    36.0|    138.0|2020-01-22 00:00:00|        2|    0|        0|             Hiraide|null|
|  2|            null|           Singapore|  1.2833| 103.8333|2020-01-22 00:00:00|        0|    0|        0|           Singapore|null|
|  3|            null|               Nepal| 28.1667|    84.25|2020-01-22 00:00:00|        0|    0|        0|           Kathmandu|null|
|  4|            null|            Malaysia|     2.5|   

In [52]:
corona_df.filter("Country = 'US'").sort(col("Date"), ascending = False).show()

+-----+----------------+-------+------------------+---------+-------------------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|               Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+-------------------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20 00:00:00|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20 00:00:00|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20 00:00:00|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20 00:00:00|     1177|   23|        0|      California|      California|
|27767

In [53]:
# we can also sort using order by function , same output 
corona_df.filter("Country = 'US'").orderBy(col("Date"), ascending = False).show()

+-----+----------------+-------+------------------+---------+-------------------+---------+-----+---------+----------------+----------------+
|  _c0|           State|Country|               Lat|     Long|               Date|Confirmed|Death|Recovered|   state_cleaned|            City|
+-----+----------------+-------+------------------+---------+-------------------+---------+-----+---------+----------------+----------------+
|27764|      Washington|     US|           47.4009|-121.4905|2020-03-20 00:00:00|     1524|   83|        0|      Washington|      Washington|
|27784|         Arizona|     US|           33.7298|-111.4312|2020-03-20 00:00:00|       78|    0|        0|         Arizona|         Arizona|
|27765|        New York|     US|           42.1657| -74.9481|2020-03-20 00:00:00|     8310|   42|        0|        New York|        New York|
|27766|      California|     US|           36.1162|-119.6816|2020-03-20 00:00:00|     1177|   23|        0|      California|      California|
|27767

Sort & group by are wide transformations & hence they are expensive. 
If we know which partition have which data than we can use sort within partions. 


In [54]:
corona_df.filter("Country = 'US'").sortWithinPartitions([col("Date"), col("Confirmed")], ascending = False).show()
# This is basically 
# For optimization purposes, it’s sometimes advisable to sort within each partition before another
# set of transformations.

+-----+--------------+-------+-------+---------+-------------------+---------+-----+---------+--------------+--------------+
|  _c0|         State|Country|    Lat|     Long|               Date|Confirmed|Death|Recovered| state_cleaned|          City|
+-----+--------------+-------+-------+---------+-------------------+---------+-----+---------+--------------+--------------+
|27765|      New York|     US|42.1657| -74.9481|2020-03-20 00:00:00|     8310|   42|        0|      New York|      New York|
|27764|    Washington|     US|47.4009|-121.4905|2020-03-20 00:00:00|     1524|   83|        0|    Washington|    Washington|
|27766|    California|     US|36.1162|-119.6816|2020-03-20 00:00:00|     1177|   23|        0|    California|    California|
|27773|    New Jersey|     US|40.2989|  -74.521|2020-03-20 00:00:00|      890|   11|        0|    New Jersey|    New Jersey|
|27776|      Illinois|     US|40.3495| -88.9861|2020-03-20 00:00:00|      585|    5|        0|      Illinois|      Illinois|


In [55]:
corona_df.describe().show()

+-------+-----------------+---------+-----------+------------------+------------------+------------------+------------------+------------------+-------------+-----------+
|summary|              _c0|    State|    Country|               Lat|              Long|         Confirmed|             Death|         Recovered|state_cleaned|       City|
+-------+-----------------+---------+-----------+------------------+------------------+------------------+------------------+------------------+-------------+-----------+
|  count|            28143|    19116|      28143|             28143|             28143|             28143|             28143|             28143|        28143|      14573|
|   mean|          14071.0|     null|       null|30.965553459118834|-34.57031257861667|161.88245744945456| 5.494368048893153| 60.17290267562094|         null|       null|
| stddev|8124.328649186959|     null|       null|19.365472826597646| 80.78375872452575| 2519.847217725942|109.29475709869875|1346.6597829124426| 

In [56]:
corona_df.printSchema()
# Since we have done infer schema this infromation is inferred in that step. 

root
 |-- _c0: integer (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Death: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- state_cleaned: string (nullable = true)
 |-- City: string (nullable = true)



In [58]:
corona_df.filter('Confirmed>10000').sort(col('Confirmed')).show()
# By default sort is ascending 

+-----+------+-------+-------+--------+-------------------+---------+-----+---------+-------------+----+
|  _c0| State|Country|    Lat|    Long|               Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+------+-------+-------+--------+-------------------+---------+-----+---------+-------------+----+
|24005|  null|   Iran|   32.0|    53.0|2020-03-12 00:00:00|    10075|  429|     2959|       Tehran|null|
|22912|  null|  Italy|   43.0|    12.0|2020-03-10 00:00:00|    10149|  631|      724|         Rome|null|
|27346|France| France|46.2276|  2.2137|2020-03-19 00:00:00|    10871|  243|       12|       France|null|
| 5401| Hubei|  China|30.9756|112.2707|2020-02-02 00:00:00|    11177|  350|      295|        Hubei|null|
|24482|  null|   Iran|   32.0|    53.0|2020-03-13 00:00:00|    11364|  514|     2959|       Tehran|null|
|26253|  null|  Spain|   40.0|    -4.0|2020-03-17 00:00:00|    11748|  533|     1028|       Toledo|null|
|26723|  null|Germany|   51.0|     9.0|2020-03-18 00:00

In [59]:
corona_df.agg({"Date":"max"}).collect()

[Row(max(Date)=datetime.datetime(2020, 3, 20, 0, 0))]

In [60]:
# Can also get aggregations on multiple columns 

corona_df.agg({"Date":"max", "Confirmed": "max"}).collect()

[Row(max(Confirmed)=67800, max(Date)=datetime.datetime(2020, 3, 20, 0, 0))]

In [62]:
# Based on this lets get the latest date data frame. 
# get the latest date 
max_date = corona_df.agg({"Date":"max"})
max_date.show()

+-------------------+
|          max(Date)|
+-------------------+
|2020-03-20 00:00:00|
+-------------------+



In [66]:
import pyspark.sql.functions as F
corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")).show()

+--------------+--------------------+-------------------+
|       Country|       State_cleaned|               Date|
+--------------+--------------------+-------------------+
|      Cameroon|             Yaounde|2020-03-20 00:00:00|
|        Cyprus|             Nicosia|2020-03-20 00:00:00|
|            US|            Michigan|2020-03-20 00:00:00|
|         China|             Qinghai|2020-03-20 00:00:00|
|      Portugal|              Lisbon|2020-03-20 00:00:00|
|            US|            Colorado|2020-03-20 00:00:00|
|United Kingdom|      Cayman Islands|2020-03-20 00:00:00|
|            US|            Missouri|2020-03-20 00:00:00|
|         China|              Hainan|2020-03-20 00:00:00|
|     Australia|Australian Capita...|2020-03-20 00:00:00|
|            US|                Guam|2020-03-20 00:00:00|
|        France|             Reunion|2020-03-20 00:00:00|
|      Colombia|        Cundinamarca|2020-03-20 00:00:00|
|          Cuba|              Havana|2020-03-20 00:00:00|
|     Mauritiu

In [67]:
# Using the above commandm we can join the original dataset with the above created dataset which gives us the maximum date for country,statecleaned group
# Joining this will give us the latest data & full  for this group

corona_df.join(
    corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")), 
    on = ['Country', 'State_cleaned', 'Date'], 
    how =  "inner"
).show()

+--------------------+--------------------+-------------------+-----+----------------+--------+---------+---------+-----+---------+----+
|             Country|       state_cleaned|               Date|  _c0|           State|     Lat|     Long|Confirmed|Death|Recovered|City|
+--------------------+--------------------+-------------------+-----+----------------+--------+---------+---------+-----+---------+----+
|            Thailand|             Bangkok|2020-03-20 00:00:00|27666|            null|    15.0|    101.0|      322|    1|       42|null|
|               Japan|             Hiraide|2020-03-20 00:00:00|27667|            null|    36.0|    138.0|      963|   33|      191|null|
|           Singapore|           Singapore|2020-03-20 00:00:00|27668|            null|  1.2833| 103.8333|      385|    0|      124|null|
|               Nepal|           Kathmandu|2020-03-20 00:00:00|27669|            null| 28.1667|    84.25|        1|    0|        1|null|
|            Malaysia|             Sarawa

In [69]:
# Lets sort the above query with respect to confirmed cases. 
corona_df.join(
    corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")), 
    on = ['Country', 'State_cleaned', 'Date'], 
    how =  "inner"
).sort(col("Confirmed"), ascending = False).show()

+--------------+----------------+-------------------+-----+--------------+-------+-------------------+---------+-----+---------+----------+
|       Country|   state_cleaned|               Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|      City|
+--------------+----------------+-------------------+-----+--------------+-------+-------------------+---------+-----+---------+----------+
|         China|           Hubei|2020-03-20 00:00:00|27820|         Hubei|30.9756|           112.2707|    67800| 3133|    58382|      null|
|         Italy|            Rome|2020-03-20 00:00:00|27682|          null|   43.0|               12.0|    47021| 4032|     4440|      null|
|         Spain|          Toledo|2020-03-20 00:00:00|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|      null|
|       Germany|          Berlin|2020-03-20 00:00:00|27677|          null|   51.0|                9.0|    19848|   67|      180|      null|
|          Iran|    

Window Functions 

Window aggregate functions (aka window functions or windowed aggregates) are functions that perform a calculation over a group of records called window that are in some relation to the current record. 
Window functions are also called over functions due to how they are applied using over operator.
Spark SQL supports three kinds of window functions:

ranking functions ,  analytic functions , aggregate function

We are partitioning the data. We can create partitions logically. When we partition the data using country & state & than order it by date in window specification. 
Than we assign a row number over the partition. 

In [74]:
from pyspark.sql.functions import row_number 
from pyspark.sql.window import Window

# here we are partition the data by country & state & than order by date 
ws = Window().partitionBy("Country", "State_cleaned").orderBy(col("Date").desc())

# adding the column row numvber over the window specification 
corona_df.withColumn("row_num", row_number().over(ws)).show()


# so using the window function, we have logically partitioned data.
# for each date of the group it is going to assign a row number 
# we see that for the cameroin & states we have row number for each date. 

+-----+-----+--------+------------------+-------+-------------------+---------+-----+---------+-------------+----+-------+
|  _c0|State| Country|               Lat|   Long|               Date|Confirmed|Death|Recovered|state_cleaned|City|row_num|
+-----+-----+--------+------------------+-------+-------------------+---------+-----+---------+-------------+----+-------+
|27745| null|Cameroon|3.8480000000000003|11.5021|2020-03-20 00:00:00|       20|    0|        0|      Yaounde|null|      1|
|27268| null|Cameroon|3.8480000000000003|11.5021|2020-03-19 00:00:00|       13|    0|        0|      Yaounde|null|      2|
|26791| null|Cameroon|3.8480000000000003|11.5021|2020-03-18 00:00:00|       10|    0|        0|      Yaounde|null|      3|
|26314| null|Cameroon|3.8480000000000003|11.5021|2020-03-17 00:00:00|       10|    0|        0|      Yaounde|null|      4|
|25837| null|Cameroon|3.8480000000000003|11.5021|2020-03-16 00:00:00|        4|    0|        0|      Yaounde|null|      5|
|25360| null|Cam

In [75]:
# Now we are adding one more condition 
corona_df.withColumn("row_num", row_number().over(ws)).where(col("row_num") == 1).show()
# This is basically getting the records where ever the row number is 1. 
# this is ist row for each country & state 
# This is another way of filtering or we can say grouping 


+-----+--------------------+--------------+------------------+---------+-------------------+---------+-----+---------+--------------------+--------+-------+
|  _c0|               State|       Country|               Lat|     Long|               Date|Confirmed|Death|Recovered|       state_cleaned|    City|row_num|
+-----+--------------------+--------------+------------------+---------+-------------------+---------+-----+---------+--------------------+--------+-------+
|27745|                null|      Cameroon|3.8480000000000003|  11.5021|2020-03-20 00:00:00|       20|    0|        0|             Yaounde|    null|      1|
|27859|             Qinghai|         China|           35.7452|  95.9956|2020-03-20 00:00:00|       18|    0|       18|             Qinghai|    null|      1|
|27762|                null|        Cyprus|           35.1264|  33.4299|2020-03-20 00:00:00|       67|    0|        0|             Nicosia|    null|      1|
|27812|            Michigan|            US|           43.3

In [0]:
 # Moving back a bit 
 # Here we are creating the dataframe using the join condition we saw earlier
 
 
 corona_max_df = corona_df.join(
    corona_df.groupBy("Country", "State_cleaned").agg(F.max("Date").alias("Date")), 
    on = ['Country', 'State_cleaned', 'Date'], 
    how =  "inner"
).sort(col("Confirmed"), ascending = False)

In [100]:
corona_max_df.show()

+--------------+----------------+-------------------+-----+--------------+-------+-------------------+---------+-----+---------+----------+
|       Country|   state_cleaned|               Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|      City|
+--------------+----------------+-------------------+-----+--------------+-------+-------------------+---------+-----+---------+----------+
|         China|           Hubei|2020-03-20 00:00:00|27820|         Hubei|30.9756|           112.2707|    67800| 3133|    58382|      null|
|         Italy|            Rome|2020-03-20 00:00:00|27682|          null|   43.0|               12.0|    47021| 4032|     4440|      null|
|         Spain|          Toledo|2020-03-20 00:00:00|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|      null|
|       Germany|          Berlin|2020-03-20 00:00:00|27677|          null|   51.0|                9.0|    19848|   67|      180|      null|
|          Iran|    

In [80]:
# We have time series data
# Here we have grouped the data by country
# We ned to convert the date column to columsn for each date
corona_df.groupBy("Country").pivot("Date").agg(F.sum("Confirmed")).show()

+-----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------

In [82]:
# Use less here to use cross tab 
corona_df.filter("Country == 'US'").crosstab("State", "Date").show()

+-------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+-----------

In [85]:
# Using the max tab 
# lets see the confirmed cases for the latest 
corona_max_df.groupBy("Country").agg({'Confirmed': 'sum'}).show()

+-----------+--------------+
|    Country|sum(Confirmed)|
+-----------+--------------+
|       Chad|             1|
|     Russia|           253|
|   Paraguay|            13|
|    Senegal|            38|
|     Sweden|          1639|
| Cabo Verde|             1|
|     Guyana|             7|
|Philippines|           230|
|   Djibouti|             1|
|   Malaysia|          1030|
|  Singapore|           385|
|       Fiji|             1|
|     Turkey|           359|
|       Iraq|           208|
|    Germany|         19848|
|   Cambodia|            51|
|Afghanistan|            24|
|     Jordan|            85|
|     Rwanda|            17|
|   Maldives|            13|
+-----------+--------------+
only showing top 20 rows



In [87]:
corona_max_df.groupBy("Country").agg({'Confirmed': 'sum', 'Recovered': 'sum','Death':'sum'}).orderBy("sum(Confirmed)", ascending = False).show()
# Adding multiple cases 
# ordering by confirmed cases 


+--------------+--------------+----------+--------------+
|       Country|sum(Recovered)|sum(Death)|sum(Confirmed)|
+--------------+--------------+----------+--------------+
|         China|         71266|      3253|         81250|
|         Italy|          4440|      4032|         47021|
|         Spain|          1588|      1043|         20410|
|       Germany|           180|        67|         19848|
|          Iran|          6745|      1433|         19644|
|            US|             0|       244|         19100|
|        France|            12|       450|         12726|
|  Korea, South|          1540|        94|          8652|
|   Switzerland|            15|        54|          5294|
|United Kingdom|            67|       178|          4014|
|   Netherlands|             2|       107|          3003|
|       Austria|             9|         6|          2388|
|       Belgium|             1|        37|          2257|
|        Norway|             1|         7|          1914|
|        Swede

In [88]:
# We can filter the main dataframe on one country as well. 
corona_df.filter("Country == 'Italy'").sort("Date", ascending = False).show()

+-----+-----+-------+----+----+-------------------+---------+-----+---------+-------------+----+
|  _c0|State|Country| Lat|Long|               Date|Confirmed|Death|Recovered|state_cleaned|City|
+-----+-----+-------+----+----+-------------------+---------+-----+---------+-------------+----+
|27682| null|  Italy|43.0|12.0|2020-03-20 00:00:00|    47021| 4032|     4440|         Rome|null|
|27205| null|  Italy|43.0|12.0|2020-03-19 00:00:00|    41035| 3405|     4440|         Rome|null|
|26728| null|  Italy|43.0|12.0|2020-03-18 00:00:00|    35713| 2978|     4025|         Rome|null|
|26251| null|  Italy|43.0|12.0|2020-03-17 00:00:00|    31506| 2503|     2941|         Rome|null|
|25774| null|  Italy|43.0|12.0|2020-03-16 00:00:00|    27980| 2158|     2749|         Rome|null|
|25297| null|  Italy|43.0|12.0|2020-03-15 00:00:00|    24747| 1809|     2335|         Rome|null|
|24820| null|  Italy|43.0|12.0|2020-03-14 00:00:00|    21157| 1441|     1966|         Rome|null|
|24343| null|  Italy|43.0|12.0

In [0]:
# using withColumns as Active 
corona_max_df = corona_max_df.withColumn("Active", corona_max_df.Confirmed - corona_max_df.Recovered - corona_max_df.Death).sort("Active", ascending = False)

In [106]:
corona_max_df.show()

+--------------+----------------+-------------------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|       Country|   state_cleaned|               Date|  _c0|         State|    Lat|               Long|Confirmed|Death|Recovered|      City|Active|
+--------------+----------------+-------------------+-----+--------------+-------+-------------------+---------+-----+---------+----------+------+
|         Italy|            Rome|2020-03-20 00:00:00|27682|          null|   43.0|               12.0|    47021| 4032|     4440|      null| 38549|
|       Germany|          Berlin|2020-03-20 00:00:00|27677|          null|   51.0|                9.0|    19848|   67|      180|      null| 19601|
|         Spain|          Toledo|2020-03-20 00:00:00|27684|          null|   40.0|               -4.0|    20410| 1043|     1588|      null| 17779|
|        France|          France|2020-03-20 00:00:00|27823|        France|46.2276|             2.2137|    12612|  450|

In [107]:
corona_max_df.groupBy("Country").sum("Active").orderBy("sum(Active)", ascending = False).show()

+--------------+-----------+
|       Country|sum(Active)|
+--------------+-----------+
|         Italy|      38549|
|       Germany|      19601|
|            US|      18856|
|         Spain|      17779|
|        France|      12264|
|          Iran|      11466|
|  Korea, South|       7018|
|         China|       6731|
|   Switzerland|       5225|
|United Kingdom|       3769|
|   Netherlands|       2894|
|       Austria|       2373|
|       Belgium|       2219|
|        Norway|       1906|
|        Sweden|       1607|
|       Denmark|       1327|
|      Portugal|       1009|
|      Malaysia|        940|
|        Canada|        922|
|       Czechia|        829|
+--------------+-----------+
only showing top 20 rows

